In [1]:
import pandas as pd
import requests
import json
from requests.auth import HTTPBasicAuth
from ibm_watson import AssistantV2
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson import ApiException

In [2]:
API_KEY = ""
SERVICE_URL = "https://api.eu-de.assistant.watson.cloud.ibm.com"
API_APP_URL = "https://api.eu-de.assistant.watson.cloud.ibm.com/instances/6833047d-680e-4bd1-ace3-2a9e87387e14"
API_ASSISTANT_ID = "bfe2fe18-c68e-4f54-9891-7bbcc978c76a"
API_VERSION = "2021-11-27"

In [17]:
test    = pd.read_csv('./testset_5.csv')

In [18]:
authenticator = IAMAuthenticator(API_KEY)
assistant = AssistantV2(
    version=API_VERSION,
    authenticator = authenticator
)

assistant.set_service_url(SERVICE_URL)
try:
    response = assistant.create_session(
        assistant_id=API_ASSISTANT_ID
    ).get_result()
except ApiException as ex:
    print("Method failed with status code " + str(ex.code) + ": " + ex.message)

session_id = json.dumps(response['session_id'])
print(session_id)

"1195b793-25a2-45d0-9d12-f65779d4651c"


In [ ]:
example_count = test.shape[0]
correctly_detected = 0
incorrects = []

for i in range(example_count):
    query = test['Query'][i]
    ground_true = test['Category'][i].replace(' ', '_')
    print(query, ground_true)
    try:
        response = assistant.message(
            assistant_id=API_ASSISTANT_ID,
            session_id=session_id.strip('"'),
            input={
                'message_type': 'text',
                'text': query
            }
        ).get_result()

        answer = json.dumps(response['output']['generic'][0]['text'])
        if len(response["output"]["intents"]) > 0:
            intent = json.dumps(response['output']['intents'][0]['intent'])
            confidence = json.dumps(response['output']['intents'][0]['confidence'])
            print(intent, confidence)    
            if(str(intent).strip('"') == ground_true):
                correctly_detected = correctly_detected + 1
            else:
                incorrects.append(str(query + ',' + intent + ',' + ground_true))
        else:
            incorrects.append(str(query + ',[]' + ',' + ground_true))
    except ApiException as ex:
        print("Method failed with status code " + str(ex.code) + ": " + ex.message)



In [ ]:
print('Number of test examples: ', example_count)
print('Correctly detected: ', correctly_detected)
print('Incorrectly detected: ', example_count - correctly_detected)
print('Accuracy: ', correctly_detected / example_count)
print('Incorrectly_detected: ', incorrects)